## Classifying CIFAR-10 with Data Augmentation

In this exercise, we revisit CIFAR-10 and the networks we previously built.  We will use real-time data augmentation to try to improve our results.

When you are done going through the notebook, experiment with different data augmentation parameters and see if they help (or hurt!) the performance of your classifier.

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In Exercise 6, we built two models.  One was smaller (181K parameters) while the second was larger (1.25M) parameters.  Below we use the smaller model and train it with data augmentation.

In [5]:
# Let's build a CNN using Keras' Sequential capabilities

model_1 = Sequential()


## 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5), strides = (2,2), padding='same',
                 input_shape=x_train.shape[1:]))
model_1.add(Activation('relu'))

## Another 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5), strides = (2,2)))
model_1.add(Activation('relu'))

## 2x2 max pooling reduces to 3 x 3 x 32
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Dropout(0.25))

## Flatten turns 3x3x32 into 288x1
model_1.add(Flatten())
model_1.add(Dense(512))
model_1.add(Activation('relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(num_classes))
model_1.add(Activation('softmax'))

model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        2432      
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          25632     
_________________________________________________________________
activation_2 (Activation)    (None, 6, 6, 32)          0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 3, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 288)               0         
__________

We still have 181K parameters, even though this is a "small" model.


In [6]:
batch_size = 32

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0005, decay=1e-6)

# Let's train the model using RMSprop
model_1.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Here we define the `ImageDataGenerator` that we will use to serve images to our model during the training process.  Currently, it is configured to do some shifting and horizontal flipping.

In [7]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images
                   
datagen.fit(x_train)      # This computes any statistics that may be needed (e.g. for centering) from the training set.

    
# Fit the model on the batches generated by datagen.flow().
model_1.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=15,
                    validation_data=(x_test, y_test))

Epoch 1/15
1562/1562 [==============================] - 42s 27ms/step - loss: 1.8197 - acc: 0.3363 - val_loss: 1.4811 - val_acc: 0.4510
Epoch 2/15
1562/1562 [==============================] - 49s 32ms/step - loss: 1.5840 - acc: 0.4277 - val_loss: 1.3514 - val_acc: 0.5065
Epoch 3/15
1562/1562 [==============================] - 46s 30ms/step - loss: 1.4940 - acc: 0.4636 - val_loss: 1.3178 - val_acc: 0.5231
Epoch 4/15
1562/1562 [==============================] - 46s 29ms/step - loss: 1.4397 - acc: 0.4833 - val_loss: 1.2475 - val_acc: 0.5536
Epoch 5/15
1562/1562 [==============================] - 46s 30ms/step - loss: 1.4073 - acc: 0.4970 - val_loss: 1.2242 - val_acc: 0.5624
Epoch 6/15
1562/1562 [==============================] - 46s 30ms/step - loss: 1.3844 - acc: 0.5104 - val_loss: 1.2074 - val_acc: 0.5801
Epoch 7/15
1562/1562 [==============================] - 47s 30ms/step - loss: 1.3663 - acc: 0.5179 - val_loss: 1.1453 - val_acc: 0.6011
Epoch 8/15
1562/1562 [==========================

How does the performance compare with the non-augmented training?

In [10]:
model_1b = Sequential()
model_1b.add(Conv2D(32, (5, 5), strides = (2,2), padding='same',
                 input_shape=x_train.shape[1:]))
model_1b.add(Activation('relu'))
model_1b.add(Conv2D(32, (5, 5), strides = (2,2)))
model_1b.add(Activation('relu'))
model_1b.add(MaxPooling2D(pool_size=(2, 2)))
model_1b.add(Dropout(0.25))
model_1b.add(Flatten())
model_1b.add(Dense(512))
model_1b.add(Activation('relu'))
model_1b.add(Dropout(0.5))
model_1b.add(Dense(num_classes))
model_1b.add(Activation('softmax'))
model_1b.compile(loss='categorical_crossentropy',
                 optimizer=opt,
                 metrics=['accuracy'])

datagen_b = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.0,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.0,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images
                   
datagen_b.fit(x_train)      # This computes any statistics that may be needed (e.g. for centering) from the training set.

    
# Fit the model on the batches generated by datagen.flow().
model_1b.fit_generator(datagen_b.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=8,
                    validation_data=(x_test, y_test))

Epoch 1/8
1562/1562 [==============================] - 47s 30ms/step - loss: 1.7990 - acc: 0.3406 - val_loss: 1.5096 - val_acc: 0.4606
Epoch 2/8
1562/1562 [==============================] - 58s 37ms/step - loss: 1.5225 - acc: 0.4488 - val_loss: 1.3486 - val_acc: 0.5149
Epoch 3/8
1562/1562 [==============================] - 39s 25ms/step - loss: 1.4085 - acc: 0.4960 - val_loss: 1.3050 - val_acc: 0.5413
Epoch 4/8
1562/1562 [==============================] - 40s 25ms/step - loss: 1.3367 - acc: 0.5242 - val_loss: 1.1917 - val_acc: 0.5780
Epoch 5/8
1562/1562 [==============================] - 32s 20ms/step - loss: 1.2962 - acc: 0.5433 - val_loss: 1.1528 - val_acc: 0.5944
Epoch 6/8
1562/1562 [==============================] - 33s 21ms/step - loss: 1.2725 - acc: 0.5553 - val_loss: 1.2614 - val_acc: 0.5805
Epoch 7/8
1562/1562 [==============================] - 36s 23ms/step - loss: 1.2570 - acc: 0.5616 - val_loss: 1.1331 - val_acc: 0.6068
Epoch 8/8
1562/1562 [==============================] - 

## Exercise
### Your Turn

1. Experiment above with different settings of the data augmentation parameters.  Can you make the model do better?  Can you make it do worse?

2. As in Exercise 6, Build a more complicated model with the following pattern:
    - Conv -> Conv -> MaxPool -> Conv -> Conv -> MaxPool -> (Flatten) -> Dense -> Final Classification
    - Use strides of 1 for all convolutional layers.

3. Use data augmentation to train this model.  Can you get better performance?

In [13]:
# Let's build a CNN using Keras' Sequential capabilities

# Please provide your code here


model_2 = Sequential()
model_2.add(Conv2D(32, (5, 5), strides = (1, 1), padding='same',
                 input_shape=x_train.shape[1:], activation='relu'))
model_2.add(Conv2D(32, (5, 5), strides = (1, 1), activation='relu'))
model_2.add(MaxPooling2D(pool_size=(1, 1)))
model_2.add(Conv2D(32, (3, 3), strides = (1, 1), padding='same',
                 input_shape=x_train.shape[1:], activation='relu'))
model_2.add(Conv2D(32, (3, 3), strides = (1, 1), activation='relu'))
model_2.add(MaxPooling2D(pool_size=(1, 1)))
model_2.add(Flatten())
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(num_classes, activation='softmax'))
model_2.compile(loss='categorical_crossentropy',
                 optimizer=opt,
                 metrics=['accuracy'])

model_2.summary()

datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images
                   
datagen.fit(x_train)      # This computes any statistics that may be needed (e.g. for centering) from the training set.

    
# Fit the model on the batches generated by datagen.flow().
model_2.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=8,
                    validation_data=(x_test, y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 32, 32, 32)        2432      
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 26, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 26, 26, 32)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 21632)             0         
__________

In [ ]:
## Check number of parameters (print the summary)


In [ ]:
# initiate RMSprop optimizer

# Let's train the model using RMSprop

In [ ]:
# Compute quantities required for feature-wise normalization


# Fit the model on the batches generated by datagen.flow().

In [14]:
# Let's build a CNN using Keras' Sequential capabilities

# Please provide your code here


model_2b = Sequential()
model_2b.add(Conv2D(32, (5, 5), strides = (1, 1), padding='same',
                 input_shape=x_train.shape[1:], activation='relu'))
model_2b.add(Conv2D(32, (5, 5), strides = (1, 1), activation='relu'))
model_2b.add(MaxPooling2D(pool_size=(1, 1)))
model_2b.add(Conv2D(32, (3, 3), strides = (1, 1), padding='same',
                 input_shape=x_train.shape[1:], activation='relu'))
model_2b.add(Conv2D(32, (3, 3), strides = (1, 1), activation='relu'))
model_2b.add(MaxPooling2D(pool_size=(1, 1)))
model_2b.add(Flatten())
model_2b.add(Dense(512, activation='relu'))
model_2b.add(Dropout(0.5))
model_2b.add(Dense(num_classes, activation='softmax'))
model_2b.compile(loss='categorical_crossentropy',
                 optimizer=opt,
                 metrics=['accuracy'])

model_2b.summary()

datagen_b = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.0,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.0,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images
                   
datagen_b.fit(x_train)      # This computes any statistics that may be needed (e.g. for centering) from the training set.

    
# Fit the model on the batches generated by datagen.flow().
model_2b.fit_generator(datagen_b.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=4,
                    validation_data=(x_test, y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 32, 32, 32)        2432      
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 26, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 26, 26, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 21632)             0         
__________

In [15]:
model_2b.fit_generator(datagen_b.flow(x_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=2,
                    validation_data=(x_test, y_test))

Epoch 1/2
1562/1562 [==============================] - 558s 357ms/step - loss: 0.9046 - acc: 0.6946 - val_loss: 1.1325 - val_acc: 0.6584
Epoch 2/2
1562/1562 [==============================] - 546s 350ms/step - loss: 0.8659 - acc: 0.7119 - val_loss: 1.0586 - val_acc: 0.6412
